Md. Tanvir Jawad <br>ID: 21101221

In [1]:
try:
    import pint
except ImportError:
    !pip install pint

In [2]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
         'modsim.py')

In [3]:
from modsim import *

In [4]:
download('https://raw.githubusercontent.com/AllenDowney/' +
         'ModSimPy/master/data/World_population_estimates.html')

In [5]:
from pandas import read_html

filename = 'World_population_estimates.html'
tables = read_html(filename, header=0, index_col=0, decimal='M')
table2 = tables[2]
table2.columns = ['census', 'prb', 'un', 'maddison', 'hyde', 'tanton', 'biraben', 'mj', 'thomlinson', 'durand', 'clark']

In [6]:
un = table2.un / 1e9
census = table2.census / 1e9

In [7]:
download('https://github.com/AllenDowney/ModSimPy/raw/master/' +'chap06.py')

In [8]:
from chap06 import run_simulation

def plot_estimates():
    census.plot(style=':', label='US Census')
    un.plot(style='--', label='UN DESA')
    decorate(xlabel='Year', ylabel='World population (billions)')

In [9]:
def growth_func_quad(t, pop, system):
    return system.alpha * pop + system.beta * pop**2

In [10]:
t_0 = census.index[0]
p_0 = census[t_0]

system = System(t_0 = t_0,
                p_0 = p_0,
                alpha = 25 / 1000,
                beta = -1.8 / 1000,
                t_end = 2100)

In [18]:
results = run_simulation(system, growth_func_quad)

In [11]:
from sympy import symbols, Function, diff, Eq, dsolve, solve

t = symbols('t')
b = Function('b')(t)
alpha, beta, b_0 = symbols('alpha beta b_0')

In [12]:
eq_bacteria = Eq(diff(b, t), alpha * b + beta * b**2)


In [13]:
solution_bacteria = dsolve(eq_bacteria, b)


In [14]:
particular_solution_bacteria = solve(solution_bacteria.subs({b.subs(t, 0): b_0}), dict=True)
particular_solution_bacteria = particular_solution_bacteria[0]


In [15]:
def growth_rate_func(t):
    intercept = 0.02
    slope = -0.00021
    return intercept + slope * (t - 1970)

In [16]:
t_0_bacteria = 1960
t_end_bacteria = 2100
initial_bacterial_count = 21101221
b_0_bacteria = initial_bacterial_count

system_bacteria = System(t_0=t_0_bacteria, t_end=t_end_bacteria, b_0=b_0_bacteria, alpha_func=growth_rate_func)


In [19]:
# Corrected system_bacteria object with 'b_0_bacteria' for initial bacterial count
system_bacteria = System(t_0=t_0_bacteria, t_end=t_end_bacteria, b_0_bacteria=b_0_bacteria, alpha_func=growth_rate_func)

# Corrected function name growth_func_bacteria
def growth_func_bacteria(t, pop, system):
    return system.alpha_func(t) * pop

results_bacteria = run_simulation(system_bacteria, growth_func_bacteria)


AttributeError: ignored